In [160]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [161]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(chrome_options=chrome_options,
                          executable_path='chromedriver.exe')

fname = 'routes.txt'
with open(fname, encoding='utf8') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
urls = [x.strip() for x in content if not x.startswith('#')]

columns=['團型名稱', '團號', '價錢', '出國日期', '抵台日期',
         '剩餘空位', '坐位總數', '出國航空', '出國時間', '出國機場',
         '抵達時間', '抵達機場', '回國航空', '回國時間', '回國機場',
         '抵台時間', '抵台機場', '網址']
df = pd.DataFrame(data=None, columns=columns)

In [162]:
for index, url in enumerate(urls):
    print(f"{index+1}/{len(urls)}")
    driver.get(url)
    time.sleep(5)
    elem = driver.find_element_by_class_name("tourTopInfo")
    source_code = elem.get_attribute('innerHTML')
    soup = BeautifulSoup(source_code, 'html.parser')

    # 團型名稱
    JSON = re.compile('ProductName: ".*?"', re.DOTALL)
    trip_name = JSON.search(source_code).group(0).split()[1]

    # 團號
    trip_number = soup.find_all('th', {'class':'tptitle'})[0].span.text

    # 價錢
    price = soup.find('span', {'class':'price'}).text

    tpinfos = soup.find_all('tr', {'class':'tpinfo'})

    # 出發日期
    start_date = tpinfos[0].find_all('td')[1].text

    # 抵達日期
    back_date = tpinfos[1].find_all('td')[1].text

    # 團位數量
    empty_seats = tpinfos[2].find_all('span')[0].text
    total_seats = tpinfos[2].find_all('span')[1].text

    # 出發航空
    start_air = tpinfos[3].find('span').text

    # 出發時間, 出發機場, 抵達時間, 抵達機場
    goto_start_time, goto_start_airport, _, goto_arrival_airport, goto_arrival_time = tpinfos[4].find_all('td')[1].text.split()

    # 回國航空
    start_air = tpinfos[5].find('span').text

    # 出發時間, 出發機場, 抵達時間, 抵達機場
    back_start_time, back_start_airport, _, back_arrival_airport, back_arrival_time = tpinfos[6].find_all('td')[1].text.split()

    row = [trip_name, trip_number, price, start_date, back_date,
          empty_seats, total_seats, start_air, goto_start_time,
          goto_start_airport, goto_arrival_time, goto_arrival_airport,
          start_air, back_start_time, back_start_airport, back_arrival_time,
          back_arrival_airport, url]
    row_data=pd.Series(row,columns)
    df = df.append(row_data, ignore_index=True)
driver.close()
driver.quit()

1/29
2/29
3/29
4/29
5/29
6/29
7/29
8/29
9/29
10/29
11/29
12/29
13/29
14/29
15/29
16/29
17/29
18/29
19/29
20/29
21/29
22/29
23/29
24/29
25/29
26/29
27/29
28/29
29/29


In [154]:
driver.close()
driver.quit()

In [163]:
df.to_csv('Osaka.csv', index=False)